In [7]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-12
2024-12-13


In [8]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 316
SH600570 800
SH603986 300
SZ002459 1300
SZ002129 2000
SZ000002 2300
SH600104 1100
SH600588 1500
SH600233 1000
SZ000999 300
'''

In [9]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

[1928968:MainThread](2024-12-16 21:47:40,304) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[1928968:MainThread](2024-12-16 21:47:40,307) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[1928968:MainThread](2024-12-16 21:47:40,308) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [10]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-13 09:00:00
{'SH600570': {'amount': 800}, 'SH603986': {'amount': 300}, 'SZ002459': {'amount': 1300}, 'SZ002129': {'amount': 2000}, 'SZ000002': {'amount': 2300}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'SZ000999': {'amount': 300}, 'cash': 316.0}
{'_settle_type': 'None', 'position': {'SZ000999': {'amount': 3640, 'price': 3.7280220985412598}, 'SH603986': {'amount': 1783, 'price': 15.636964797973633}, 'SZ002459': {'amount': 17480, 'price': 1.1490143537521362}, 'SH600233': {'amount': 7140, 'price': 2.019603729248047}, 'SH600104': {'amount': 12461, 'price': 1.654193639755249}, 'SZ000002': {'amount': 14867, 'price': 1.3010629415512085}, 'SZ002129': {'amount': 4184, 'price': 4.6313066482543945}, 'SH600588': {'amount': 7188, 'price': 2.528895139694214}, 'SH600570': {'amount': 3727, 'price': 6.463601112365723}, 'cash': 316.0, 'now_account_value': 177872.186527133}, 'init_cash': 316.0}
['SZ000999', 'SH603986', 'SZ002459', 'SH600233', 'SH60

In [11]:
def simp_eval(res_df, k, t=None, day=None):
    # print(res_df)
    if day:
        date = str(cal.iloc[-day-1,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    # top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    top_k_per_group = sorted_df.groupby(['datetime']).apply(lambda x: x.nlargest(k, 'score'))#.reset_index(level=1, drop=1)
    if t:
        # top_k_per_group = top_k_per_group.tail(t)
        top_k_per_group = top_k_per_group.groupby(['datetime']).apply(lambda x: x.nsmallest(t, 'score'))
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    s, n = top_k_per_group['win'].sum(), len(top_k_per_group)
    # print(s, n)
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win 

def run_model(model, path='/home/linq/finance/qniverse/log'):
    res_df = pd.read_csv(f'{path}/{model}_backtest_return_{TODAY}.csv', index_col=['datetime','instrument'])

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        top60ic, top60win = simp_eval(res_df, 60, 10, day=day)
        top70ic, top70win = simp_eval(res_df, 70, 10, day=day)
        top80ic, top80win = simp_eval(res_df, 80, 10, day=day)        
        top90ic, top90win = simp_eval(res_df, 90, 10, day=day)
        top100ic, top100win = simp_eval(res_df, 100, 10, day=day)
        top110ic, top110win = simp_eval(res_df, 110, 10, day=day)
        top120ic, top120win = simp_eval(res_df, 120, 10, day=day)
        top130ic, top130win = simp_eval(res_df, 130, 10, day=day)
        top140ic, top140win = simp_eval(res_df, 140, 10, day=day)
        top150ic, top150win = simp_eval(res_df, 150, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win, top60win, top70win, top80win, top90win, top100win, top110win, top120win, top130win, top140win, top150win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic, top60ic, top70ic, top80ic, top90ic, top100ic, top110ic, top120ic, top130ic, top140ic, top150ic]
    day1metrics = eval_day(2)
    day5metrics = eval_day(6)
    day10metrics = eval_day(11)
    day20metrics = eval_day(21)
    
    pred_df = pd.read_csv(f'{path}/{model}_tomorrow_return_{TODAY}.csv', index_col=['datetime','instrument'])

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df

# Run

In [12]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

name_lst =  ['PatchTST', 'PDF', 'SegRNN', 'TimeBridge', 'TimeMixer', 'TimesNet', 'WFTNet', 'GBDTS']
for model in name_lst:
    NAME = model
    print(model)
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    _df = pred_df.loc[:,'score']
    _df.name = model
    # print(_df)
    pred_lst.append(_df)

PatchTST
PDF
SegRNN
TimeBridge
TimeMixer
TimesNet
WFTNet
GBDTS


In [13]:
pred_df = pd.concat(pred_lst, axis=1)

# pred_df = pred_df.loc[:, ~pred_df.columns.duplicated()]
_pred_df = pred_df
pred_df

PatchTST       PDF    SegRNN  TimeBridge  TimeMixer  \
datetime   instrument                                                        
2024-12-13 SH600372    0.003669  0.002284 -0.003079   -0.003971   0.002654   
           SZ300442    0.002449  0.001945 -0.003659   -0.004765  -0.000481   
           SH601012    0.002135  0.000209 -0.003096   -0.003520   0.003428   
           SH600489    0.002130  0.001931 -0.001204   -0.006783  -0.001870   
           SZ000895    0.002071 -0.001195 -0.001390   -0.011623   0.003924   
...                         ...       ...       ...         ...        ...   
           SZ000408    0.000272 -0.003948 -0.000796   -0.003647  -0.000459   
           SZ002648    0.000247 -0.002400  0.000010   -0.010515   0.007589   
           SH601021    0.000154 -0.000691 -0.002266   -0.011558  -0.002216   
           SH600938    0.000117 -0.003377 -0.001821   -0.002919   0.007674   
           SH601699    0.000106 -0.003494 -0.000742    0.000405   0.001764   

                       TimesNet    WFTNet         GBDTS  
datetime   instrument                                    
2024-12-13 SH600372    0.004009  0.001263  4.310966e-06  
           SZ300442    0.007775  0.002849 -3.275044e-01  
           SH601012    0.012360 -0.002906  5.557712e-07  
           SH600489    0.009178 -0.000600  2.094552e-02  
           SZ000895   -0.005087  0.007658  3.671730e-02  
...                         ...       ...           ...  
           SZ000408   -0.012367 -0.007672  2.493195e-02  
           SZ002648   -0.051195 -0.006040  2.964805e-02  
           SH601021   -0.015378 -0.016518  2.291845e-02  
           SH600938   -0.004270 -0.001136  2.269940e-02  
           SH601699    0.019261 -0.007506  4.542416e-03  

[312 rows x 8 columns]

In [14]:

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([_pred_df[n]*info_win_dict5[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [15]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df

In [16]:
hold_lst

['SZ000999',
 'SH603986',
 'SZ002459',
 'SH600233',
 'SH600104',
 'SZ000002',
 'SZ002129',
 'SH600588',
 'SH600570']

In [17]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [18]:
color_lst = [n for n in report_pos.columns if n not in ['close']]
# color_lst

In [19]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,PatchTST,PDF,SegRNN,TimeBridge,TimeMixer,TimesNet,WFTNet,GBDTS,mean,std,score,close
instrument,,,,,,,,,,,,
SH603986,0.000990,0.002814,-0.001478,-0.005309,-0.006301,0.020019,0.002858,-0.113690,-0.012512,0.038989,-0.041919,92.979996
SH600104,0.000835,0.002128,-0.002302,-0.004241,0.004030,-0.012122,-0.013623,-0.053344,-0.009830,0.017500,-0.034308,18.740000
SZ000999,0.000773,0.002244,-0.001645,-0.007163,-0.000406,-0.011770,-0.005742,0.011175,-0.001567,0.006486,-0.005174,45.239998
SZ002459,0.001906,-0.003905,-0.001287,-0.008618,0.003175,-0.000111,0.019796,0.000000,0.001370,0.007760,0.007177,15.450000
SH600588,0.000894,0.002134,-0.001058,-0.003308,0.001420,0.024545,-0.000574,0.000000,0.003007,0.008291,0.009838,12.120000
SZ000002,0.001155,0.002271,-0.002173,0.001648,-0.001367,0.024070,-0.000650,0.000000,0.003119,0.008045,0.009968,8.410000
SZ002129,0.001157,-0.001570,-0.002179,0.002603,0.000847,0.035117,-0.007176,0.000001,0.003600,0.012232,0.010465,9.690001
SH600233,0.001375,0.002211,-0.000842,-0.002351,0.003652,0.009003,-0.002213,0.017880,0.003589,0.006412,0.011801,14.419999
SH600570,0.001847,-0.001964,-0.000591,-0.003555,0.011720,0.031938,0.001168,0.000000,0.005070,0.011030,0.016511,30.120001


In [20]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top5 = report_top.head(60).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top6 = report_top.head(70).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top7 = report_top.head(80).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top8 = report_top.head(90).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top9 = report_top.head(100).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top10 = report_top.head(110).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top11 = report_top.head(120).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top12 = report_top.head(130).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top13 = report_top.head(140).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top14 = report_top.head(150).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [21]:
sty_report_top0

,PatchTST,PDF,SegRNN,TimeBridge,TimeMixer,TimesNet,WFTNet,GBDTS,mean,std,score,close
instrument,,,,,,,,,,,,
SH603501,0.000893,0.004876,-0.000960,-0.013554,-0.003074,0.088676,-0.005705,0.030273,0.012678,0.031115,0.041586,98.660004
SH601600,0.001126,0.000749,-0.001032,-0.006486,0.001451,0.046192,-0.001219,0.038784,0.009946,0.019023,0.033006,7.600000
SH601006,0.001331,-0.003785,-0.000932,-0.000474,0.009728,0.036339,0.012292,0.017438,0.008992,0.012453,0.030764,6.880000
SH601818,0.001045,0.000025,-0.000629,-0.005001,-0.001365,0.041977,-0.000138,0.033609,0.008691,0.017014,0.028907,3.720000
SH603195,0.000408,-0.002416,-0.002945,-0.007878,0.003981,0.026187,0.004555,0.039771,0.007708,0.015434,0.026506,72.899994
SZ002027,0.001237,0.002543,-0.000236,-0.004064,0.001918,0.033438,-0.002792,0.026044,0.007261,0.013279,0.023839,6.950000
SH600958,0.000635,0.004304,-0.001374,0.004270,0.003473,0.020760,0.000872,0.022982,0.006990,0.008804,0.023073,10.840000
SH600845,0.001036,0.004685,-0.001026,0.000139,0.009903,0.028916,-0.006654,0.020398,0.007175,0.011219,0.022813,27.680000
SH600887,0.001158,0.000488,-0.000758,-0.001556,-0.003413,0.043239,-0.009436,0.028171,0.007237,0.017137,0.022760,29.250000


In [22]:
sty_report_top1

,PatchTST,PDF,SegRNN,TimeBridge,TimeMixer,TimesNet,WFTNet,GBDTS,mean,std,score,close
instrument,,,,,,,,,,,,
SH600893,0.001060,0.002076,-0.000887,-0.002533,-0.000925,0.026945,-0.006703,0.034420,0.006682,0.014195,0.021553,40.199997
SH688126,0.001256,0.001025,-0.000773,0.002914,-0.002778,0.013435,-0.002011,0.039318,0.006548,0.013261,0.021535,21.380001
SZ000625,0.000882,0.000997,-0.001183,0.006439,0.004175,0.018531,0.007363,0.010378,0.005948,0.005962,0.020058,14.170000
SH603260,0.001427,-0.001046,-0.001467,-0.000696,0.003095,0.026825,-0.005669,0.027531,0.006250,0.012311,0.019923,58.779995
SH600415,0.000893,-0.001653,-0.001087,-0.003751,-0.000510,0.039454,-0.004640,0.017673,0.005797,0.014321,0.018543,14.340001
SZ002714,0.000876,-0.002720,-0.001281,-0.003074,0.000612,0.037561,-0.006945,0.021681,0.005839,0.014474,0.018380,40.340000
SZ002271,0.000940,0.000430,-0.002509,0.003414,0.002737,0.013513,-0.007342,0.033878,0.005633,0.012026,0.017722,13.740001
SZ000001,0.000445,0.002055,-0.002130,-0.005279,0.000164,0.029270,0.000390,0.016799,0.005214,0.010946,0.017527,11.560000
SZ300498,0.001134,-0.001091,-0.001363,-0.002176,0.003651,0.028534,0.004866,0.007755,0.005164,0.009401,0.017465,17.440001


In [23]:
sty_report_top2

,PatchTST,PDF,SegRNN,TimeBridge,TimeMixer,TimesNet,WFTNet,GBDTS,mean,std,score,close
instrument,,,,,,,,,,,,
SH688009,0.000855,0.002398,-0.000470,-0.006660,0.000088,0.022523,-0.007109,0.029872,0.005187,0.012679,0.016795,6.350000
SZ002555,0.000528,-0.003261,-0.000298,-0.004951,0.010550,0.018357,-0.004407,0.024720,0.005155,0.010617,0.016606,17.090000
SH601916,0.001637,0.005250,-0.000929,-0.007574,-0.000074,0.032195,-0.007166,0.017856,0.005149,0.012664,0.016561,2.950000
SH600570,0.001847,-0.001964,-0.000591,-0.003555,0.011720,0.031938,0.001168,0.000000,0.005070,0.011030,0.016511,30.120001
SH601166,0.001445,0.011387,-0.000670,-0.006338,0.014286,0.014700,-0.012848,0.020336,0.005287,0.010883,0.016462,18.149998
SH601100,0.001380,-0.002659,-0.001218,0.000152,0.007906,0.013918,-0.010523,0.033045,0.005250,0.012495,0.016057,54.980000
SH603392,0.001096,-0.004258,-0.000588,-0.000796,0.006382,0.009156,0.004448,0.022054,0.004687,0.007708,0.015958,75.369995
SH600050,0.000833,0.003686,-0.002002,0.001034,-0.001909,0.013064,-0.002027,0.024119,0.004600,0.008734,0.015144,5.220000
SH600031,0.001109,0.000007,-0.001476,-0.000525,-0.002609,0.023452,-0.009703,0.028878,0.004892,0.012734,0.015104,16.559999


In [24]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

# dfi.export(infodf, './tmp/ensemble_all_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_all_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_all_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_all_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_all_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_all_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_all_top2.png',table_conversion='matplotlib')
dfi.export(sty_report_top3, './tmp/ensemble_all_top3.png',table_conversion='matplotlib')
dfi.export(sty_report_top4, './tmp/ensemble_all_top4.png',table_conversion='matplotlib')
dfi.export(sty_report_top5, './tmp/ensemble_all_top5.png',table_conversion='matplotlib')
# dfi.export(sty_report_top6, './tmp/ensemble_all_top6.png',table_conversion='matplotlib')
# dfi.export(sty_report_top7, './tmp/ensemble_all_top7.png',table_conversion='matplotlib')
# dfi.export(sty_report_top8, './tmp/ensemble_all_top8.png',table_conversion='matplotlib')
# dfi.export(sty_report_top9, './tmp/ensemble_all_top9.png',table_conversion='matplotlib')
# dfi.export(sty_report_top10, './tmp/ensemble_all_top10.png',table_conversion='matplotlib')
# dfi.export(sty_report_top11, './tmp/ensemble_all_top11.png',table_conversion='matplotlib')
# dfi.export(sty_report_top12, './tmp/ensemble_all_top12.png',table_conversion='matplotlib')
# dfi.export(sty_report_top13, './tmp/ensemble_all_top13.png',table_conversion='matplotlib')
# dfi.export(sty_report_top14, './tmp/ensemble_all_top14.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_all_watch.png',table_conversion='matplotlib')

In [25]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_all_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_all_ic{k}.png',table_conversion='matplotlib')